In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle


import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD   
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents(m).json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words(G).pkl','wb'))
pickle.dump(classes,open('classes(G).pkl','wb'))

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
#training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = [item[0] for item in training]
train_y = [item[1] for item in training]
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=5, verbose=1)
model.save('chatbot_model(G).h5', hist)

print("model created")


[nltk_data] Downloading package punkt to C:\Users\SPIRO-
[nltk_data]     PYTHON\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\SPIRO-
[nltk_data]     PYTHON\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



135 documents
45 classes ['acne_1', 'acne_2', 'acne_scars_1', 'acne_scars_2', 'anti_aging_1', 'anti_aging_2', 'dark_spots_1', 'dark_spots_2', 'dermatitis_1', 'dermatitis_2', 'dry_skin_1', 'dry_skin_2', 'eczema_1', 'eczema_2', 'fungal_infections_1', 'fungal_infections_2', 'greeting', 'hives_1', 'hives_2', 'oily_skin_1', 'oily_skin_2', 'psoriasis_1', 'psoriasis_2', 'rosacea_1', 'rosacea_2', 'sensitive_skin_1', 'sensitive_skin_2', 'skin_allergies_1', 'skin_allergies_2', 'skin_cancer_1', 'skin_cancer_2', 'skin_care_routine_1', 'skin_care_routine_2', 'skin_disease_overview_1', 'skin_disease_overview_2', 'skin_health_tips_1', 'skin_health_tips_2', 'skin_infections_1', 'skin_infections_2', 'skin_rash_1', 'skin_rash_2', 'sunburn_1', 'sunburn_2', 'vitiligo_1', 'vitiligo_2']
92 unique lemmatized words ['a', 'acne', 'affect', 'aging', 'allergy', 'anti-aging', 'are', 'avoid', 'bacterial', 'best', 'can', 'cancer', 'care', 'cause', 'choose', 'common', 'condition', 'daily', 'dark', 'dermatitis', 'di

Epoch 1/300



27/27 [==============================] - 1s 2ms/step - loss: 3.8480 - accuracy: 0.0074 
Epoch 2/300
27/27 [==============================] - 0s 2ms/step - loss: 3.8423 - accuracy: 0.0370
Epoch 3/300
27/27 [==============================] - 0s 2ms/step - loss: 3.7577 - accuracy: 0.0519
Epoch 4/300
27/27 [==============================] - 0s 2ms/step - loss: 3.7196 - accuracy: 0.0741
Epoch 5/300
27/27 [==============================] - 0s 2ms/step - loss: 3.6556 - accuracy: 0.0593
Epoch 6/300
27/27 [==============================] - 0s 1ms/step - loss: 3.6165 - accuracy: 0.1111
Epoch 7/300
27/27 [==============================] - 0s 1ms/step - loss: 3.5201 - accuracy: 0.1185
Epoch 8/300
27/27 [==============================] - 0s 1ms/step - loss: 3.4668 - accuracy: 0.1111
Epoch 9/300
27/27 [==============================] - 0s 2ms/step - loss: 3.4351 - accuracy: 0.1111
Epoch 10/300
27/27 [==============================] - 0s 2ms/step - loss: 3.2601 - accuracy: 0.1630
Epoch 11/300
27/27 [

D:\ANACONDA\envs\pr\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
# !pip freeze > requirements.txt

In [3]:
# !python --version

In [4]:
#! pip install -r requirements.txt
# pip install --upgrade googletrans==4.0.0-rc1
# pip install translate